In [1]:
from __future__ import division
from __future__ import print_function

import time
import torch
import numpy as np
from numpy import argmax
import torch.nn.functional as F
from pygcn.gcnio.data import dataio
from pygcn.gcnio.util import utils
from pygcn.gcn2 import GCN
import scipy.sparse
import json
from sklearn.preprocessing import StandardScaler
import glog as log
import torch.optim as optim


In [2]:
cuda = torch.cuda.is_available()
print('cuda: %s' % cuda)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'

cuda: True


In [3]:
def load_data(prefix, normalize=True):
    adj_full = scipy.sparse.load_npz('./{}/adj_full.npz'.format(prefix))
    adj_train = scipy.sparse.load_npz('./{}/adj_train.npz'.format(prefix))
    role = json.load(open('./{}/role.json'.format(prefix)))
    feats = np.load('./{}/feats.npy'.format(prefix))
    class_map = json.load(open('./{}/class_map.json'.format(prefix)))
    class_map = {int(k):v for k,v in class_map.items()}
    assert len(class_map) == feats.shape[0]
    # ---- normalize feats ----
    train_nodes = np.array(list(set(adj_train.nonzero()[0])))
    train_feats = feats[train_nodes]
    scaler = StandardScaler()
    scaler.fit(train_feats)
    feats = scaler.transform(feats)
    # -------------------------
    return adj_full, adj_train, feats, class_map, role


def process_graph_data(adj_full, adj_train, feats, class_map, role, name):
    """
    setup vertex property map for output classes, train/val/test masks, and feats
    INPUT:
        G           graph-tool graph, full graph including training,val,testing
        feats       ndarray of shape |V|xf
        class_map   dictionary {vertex_id: class_id}
        val_nodes   index of validation nodes
        test_nodes  index of testing nodes
    OUTPUT:
        G           graph-tool graph unchanged
        role        array of size |V|, indicating 'train'/'val'/'test'
        class_arr   array of |V|x|C|, converted by class_map
        feats       array of features unchanged
    """
    num_vertices = adj_full.shape[0]
    if isinstance(list(class_map.values())[0],list):
        print("labels are list")
        num_classes = len(list(class_map.values())[0])
        class_arr = np.zeros((num_vertices, 1))
        p = 0;
        for k,v in class_map.items():
            class_arr[p] = argmax(v)
            p = p+1
    else:
        num_classes = max(class_map.values()) - min(class_map.values()) + 1
        class_arr = np.zeros((num_vertices, 1))
        for k,v in class_map.items():
            class_arr[k] = v
    if name=='flickr' or name=='reddit' or name=='ppi' or name=='amazon' or name=='yelp':
        class_arr = np.squeeze(class_arr.astype(int))
        
    return adj_full, adj_train, feats, class_arr, role

In [4]:
# make sure you use the same data splits as you generated attacks
seed = 15
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

# load original dataset (to get clean features and labels)
SMALL = False
if SMALL:
    dataset = 'polblogs'
    data = dataio.Dataset(root='/tmp/', name=dataset)
    adj, features, labels = data.adj, data.features, data.labels
    idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
    
    log.info(type(adj))
    log.info(adj.shape)
    log.info(type(features))
    log.info(features.shape)
    log.info(type(labels))
    log.info(labels.shape)
    log.info(type(idx_train))
    log.info(idx_train.shape)
    log.info(type(idx_val))
    log.info(idx_val.shape)
    log.info(type(idx_test))
    log.info(idx_test.shape)
else:
    data_prefix = './dataset/ppi'
    temp_data = load_data(data_prefix)
    data_list = data_prefix.split('/')
    print(data_list[-1])
    train_data = process_graph_data(*temp_data,data_list[-1])
    adj,adj_train,features,labels,role = train_data
    features = scipy.sparse.csr_matrix(features)
    idx_train = np.array(role['tr'])
    idx_val = np.array(role['va'])
    idx_test = np.array(role['te'])
    log.info(type(adj))
    log.info(adj.shape)
    log.info(type(adj_train))
    log.info(adj_train.shape)
    log.info(type(features))
    log.info(features.shape)
    log.info(type(labels))
    log.info(labels.shape)
    log.info(type(labels[0]))
    log.info(type(idx_train))
    log.info(idx_train.shape)
    log.info(type(idx_val))
    log.info(idx_val.shape)
    log.info(type(idx_test))
    log.info(idx_test.shape)

'''
flickr: (89250,1)
ppi:    (14755,121)
reddit: (232965,1)
amazon: (1569960,107)
yelp:   (716847,100)
'''

ppi
labels are list


I0314 14:31:58.394235 18544 <ipython-input-4-4ed42d5f1dc1>:39] <class 'scipy.sparse.csr.csr_matrix'>
I0314 14:31:58.394925 18544 <ipython-input-4-4ed42d5f1dc1>:40] (14755, 14755)
I0314 14:31:58.395534 18544 <ipython-input-4-4ed42d5f1dc1>:41] <class 'scipy.sparse.csr.csr_matrix'>
I0314 14:31:58.396138 18544 <ipython-input-4-4ed42d5f1dc1>:42] (14755, 14755)
I0314 14:31:58.396625 18544 <ipython-input-4-4ed42d5f1dc1>:43] <class 'scipy.sparse.csr.csr_matrix'>
I0314 14:31:58.397109 18544 <ipython-input-4-4ed42d5f1dc1>:44] (14755, 50)
I0314 14:31:58.397601 18544 <ipython-input-4-4ed42d5f1dc1>:45] <class 'numpy.ndarray'>
I0314 14:31:58.398096 18544 <ipython-input-4-4ed42d5f1dc1>:46] (14755,)
I0314 14:31:58.398571 18544 <ipython-input-4-4ed42d5f1dc1>:47] <class 'numpy.int64'>
I0314 14:31:58.399141 18544 <ipython-input-4-4ed42d5f1dc1>:48] <class 'numpy.ndarray'>
I0314 14:31:58.399616 18544 <ipython-input-4-4ed42d5f1dc1>:49] (9716,)
I0314 14:31:58.400087 18544 <ipython-input-4-4ed42d5f1dc1>:50] <

'\nflickr: (89250,1)\nppi:    (14755,121)\nreddit: (232965,1)\namazon: (1569960,107)\nyelp:   (716847,100)\n'

In [5]:
print(labels[0])
print(labels[1])
print(labels[8])
print(labels.max())

model = GCN(nfeat=features.shape[1], nhid=32, nclass=labels.max()+1, device=device)
  
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

0
28
0
116


In [6]:
model = model.to(device)
TRAIN = 1
if TRAIN:
    model.fit(features, adj, labels, idx_train, train_iters=200, verbose=True, name='ppi')
    torch.save(model.state_dict(),'./model/gcn.pt')
TEST = 1
if TEST:
    model.load_state_dict(torch.load('./model/gcn.pt'))
    model.eval()
    model.test(idx_test)

_train_without_val
Epoch 0, training loss: 4.736241340637207
Epoch 10, training loss: 3.9234836101531982
Epoch 20, training loss: 2.5192980766296387
Epoch 30, training loss: 1.6430875062942505
Epoch 40, training loss: 1.50680673122406
Epoch 50, training loss: 1.4283138513565063
Epoch 60, training loss: 1.393007755279541
Epoch 70, training loss: 1.3790531158447266
Epoch 80, training loss: 1.3695697784423828
Epoch 90, training loss: 1.3551536798477173
Epoch 100, training loss: 1.3473656177520752
Epoch 110, training loss: 1.3355251550674438
Epoch 120, training loss: 1.3279800415039062
Epoch 130, training loss: 1.3182792663574219
Epoch 140, training loss: 1.312650203704834
Epoch 150, training loss: 1.305992841720581
Epoch 160, training loss: 1.3001456260681152
Epoch 170, training loss: 1.2979108095169067
Epoch 180, training loss: 1.293259620666504
Epoch 190, training loss: 1.2911094427108765
Forward time: 0.6792s
Layer1 time: 0.0226s
Layer1 (AX)W time: 0.0105s


ModuleAttributeError: 'GCN' object has no attribute 't_fp_relu'